### Calculate installed capacity at offshore area vs max possible

In [7]:
import pandas as pd
from constants import WIND_FARM_NODES

pd.options.mode.chained_assignment = None  # default='warn'

SPECIAL_AREAS = ["SørvestA", "SørvestD", "SørvestE"] # Areas where both techs are possible but both can't surpass area capacity

CASES = ["Base", "NOgrid", "NoHubs", "NoHubsNOgrid", "OnlyCentral", "OnlyEU", "OnlyNorth"]
#CASES = ["Base", "ConstTest"]
DF_BY_CASES_SPLIT = dict({})
DF_BY_CASES_COMBO = dict({})

GEN_TYPE_MAPPING = dict({
    "Wind_offshr_grounded": "Windoffshoregrounded",
    "Wind_offshr_floating": "Windoffshorefloating",
})

max_capacaities = pd.read_csv("../EMPIRE_extension/Output/offshore_capacities_base.csv")\
                        .rename(columns={"GeneratorTechnology": "GeneratorType", "generatorMaxInstallCapacity in MW": "Max capacity [MW]"})
max_capacaities["Node"] = max_capacaities["Node"].apply(lambda n: n.replace(" ", ""))
max_capacaities["GeneratorType"] = max_capacaities["GeneratorType"].apply(lambda g: GEN_TYPE_MAPPING[g])
max_capacaities = max_capacaities.drop(columns=["area_km2"])
max_capacaities

,Node,GeneratorType,Max capacity [MW]
0,MorayFirth,Windoffshoregrounded,5465.000000
1,FirthofForth,Windoffshoregrounded,10530.000000
2,DoggerBank,Windoffshoregrounded,16260.000000
3,Hornsea,Windoffshoregrounded,12055.000000
4,OuterDowsing,Windoffshoregrounded,6265.000000
5,Norfolk,Windoffshoregrounded,8130.000000
6,EastAnglia,Windoffshoregrounded,6755.000000
7,Borssele,Windoffshoregrounded,4881.916667
8,HollandseeKust,Windoffshoregrounded,6664.166667
9,HelgoländerBucht,Windoffshoregrounded,22910.625000


In [8]:
max_capacaities_combo = max_capacaities.copy()
max_capacaities_combo["Max capacity [MW]"] = max_capacaities_combo.groupby(["Node"])["Max capacity [MW]"].transform("sum")
max_capacaities_combo["Max capacity [MW]"] = max_capacaities_combo.apply(lambda row: row["Max capacity [MW]"] / 2 if row["Node"]\
                                                                            in SPECIAL_AREAS else row["Max capacity [MW]"], axis=1)
max_capacaities_combo = max_capacaities_combo.drop_duplicates("Node").drop(columns=["GeneratorType"]).reset_index(drop=True)
max_capacaities_combo

,Node,Max capacity [MW]
0,MorayFirth,17110.000000
1,FirthofForth,38275.000000
2,DoggerBank,16260.000000
3,Hornsea,12055.000000
4,OuterDowsing,6265.000000
5,Norfolk,8130.000000
6,EastAnglia,6755.000000
7,Borssele,4881.916667
8,HollandseeKust,6664.166667
9,HelgoländerBucht,22910.625000


In [9]:
for case in CASES:
    _df = pd.read_csv(f'Results/{case}/results_output_gen.csv')

    # Offshore nodes
    df_wind_farm_nodes_combo = _df[_df['Node'].isin(WIND_FARM_NODES)]
    df_wind_farm_nodes_split= _df[_df['Node'].isin(WIND_FARM_NODES)]

    # Sum floating + grounded cap (combo)
    df_wind_farm_nodes_combo["genInstalledCap_MW"] = df_wind_farm_nodes_combo.groupby(['Node', 'Period'])['genInstalledCap_MW'].transform('sum')
    df_wind_farm_nodes_combo = df_wind_farm_nodes_combo.drop_duplicates(("Node", "Period"), ignore_index=True)\
                                [["Node", "Period", "genInstalledCap_MW"]]
    df_wind_farm_nodes_combo = df_wind_farm_nodes_combo.merge(max_capacaities_combo, on=["Node"], how="left")
    df_wind_farm_nodes_combo["Installed capacity of max (%)"] = round(df_wind_farm_nodes_combo["genInstalledCap_MW"] \
                                                                        / df_wind_farm_nodes_combo["Max capacity [MW]"] * 100, 1)
    DF_BY_CASES_COMBO[case] = df_wind_farm_nodes_combo

    # Floating + grounded splitted
    df_wind_farm_nodes_split["genInstalledCap_MW"] = df_wind_farm_nodes_split.groupby(['Node', 'Period', 'GeneratorType'])\
                                ['genInstalledCap_MW'].transform('sum')
    df_wind_farm_nodes_split = df_wind_farm_nodes_split.drop_duplicates(("Node", "Period", 'GeneratorType'), ignore_index=True)\
                                [["Node", "Period", 'GeneratorType', "genInstalledCap_MW"]]
    df_wind_farm_nodes_split = df_wind_farm_nodes_split.merge(max_capacaities, on=["Node", 'GeneratorType'], how="left")
    df_wind_farm_nodes_split["Installed capacity of max (%)"] = round(df_wind_farm_nodes_split["genInstalledCap_MW"] \
                                                                        / df_wind_farm_nodes_split["Max capacity [MW]"] * 100, 1)
    DF_BY_CASES_SPLIT[case] = df_wind_farm_nodes_split

In [10]:
FILTER_CASE = "NOgrid"
FILTER_PERIOD = "2020-2025"
COMBO = False # If False "split" by floating/grounded; else total cap in area

filtered_df = DF_BY_CASES_COMBO[FILTER_CASE] if COMBO else DF_BY_CASES_SPLIT[FILTER_CASE]
filtered_df = filtered_df[(filtered_df["Period"] == FILTER_PERIOD) & filtered_df["Max capacity [MW]"] > 0]
filtered_df["genInstalledCap_MW"] = round(filtered_df["genInstalledCap_MW"])
filtered_df["Max capacity [MW]"] = round(filtered_df["Max capacity [MW]"])
filtered_df = filtered_df.sort_values(by="genInstalledCap_MW", ascending=False).reset_index(drop=True)
filtered_df

,Node,Period,GeneratorType,genInstalledCap_MW,Max capacity [MW],Installed capacity of max (%)
0,HelgoländerBucht,2020-2025,Windoffshoregrounded,7166.0,22911.0,31.3
1,Borssele,2020-2025,Windoffshoregrounded,3739.0,4882.0,76.6
2,EastAnglia,2020-2025,Windoffshoregrounded,2812.0,6755.0,41.6
3,OuterDowsing,2020-2025,Windoffshoregrounded,2186.0,6265.0,34.9
4,Hornsea,2020-2025,Windoffshoregrounded,1218.0,12055.0,10.1
5,Nordsøen,2020-2025,Windoffshoregrounded,1120.0,20110.0,5.6
6,MorayFirth,2020-2025,Windoffshoregrounded,588.0,5465.0,10.8
7,HollandseeKust,2020-2025,Windoffshoregrounded,357.0,6664.0,5.4
8,FirthofForth,2020-2025,Windoffshoregrounded,93.0,10530.0,0.9
9,Norfolk,2020-2025,Windoffshoregrounded,60.0,8130.0,0.7
